In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
df1 = pd.read_csv('Monday-WorkingHours.pcap_ISCX.csv')
df2 = pd.read_csv('Tuesday-WorkingHours.pcap_ISCX.csv')
df3 = pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')
df4 = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
df5 = pd.read_csv('Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
df6 = pd.read_csv('Friday-WorkingHours-Morning.pcap_ISCX.csv')
df7 = pd.read_csv('Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
df8 = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [28]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8]

print('data shape')
for i, df in enumerate(df_list, start = 1):
  rows, cols = df.shape
  print(f'df{i} -> {rows} rows, {cols} columns')

data shape
df1 -> 529918 rows, 79 columns
df2 -> 445909 rows, 79 columns
df3 -> 692703 rows, 79 columns
df4 -> 170366 rows, 79 columns
df5 -> 288602 rows, 79 columns
df6 -> 191033 rows, 79 columns
df7 -> 286467 rows, 79 columns
df8 -> 225745 rows, 79 columns


In [29]:
#모든 csv파일의 dimensions 같아 concat
df = pd.concat(df_list)
rows, cols = df.shape

print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')
df.head()

Number of rows: 2830743
Number of columns: 79
Total cells: 223628697


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [30]:
# Deleting dataframes after concating to save memory
for d in df_list: del d

In [31]:
# columns이름에서 앞뒤 공백제거
col_names = {col: col.strip() for col in df.columns}
df.rename(columns = col_names, inplace = True)

In [32]:
df.replace([np.inf, -np.inf], np.nan, inplace = True)

In [33]:
# Check for duplicate indices and reset the index if necessary
if df.index.duplicated().any():
    print("Warning: DataFrame contains duplicate indices. Resetting index.")
    df = df.reset_index(drop=True)  # Reset the index, optionally dropping the old index

In [34]:
# prompt: df의 결측치가 있는 열 출력

# 결측치가 있는 열 출력
cols_with_missing = df.columns[df.isnull().any()].tolist()
print("결측치가 있는 열:", cols_with_missing)


결측치가 있는 열: ['Flow Bytes/s', 'Flow Packets/s']


In [35]:
#Flow Bytes/s, Flow Packets/s 의 중앙값 찾기

med_flow_bytes = df['Flow Bytes/s'].median()
med_flow_packets = df['Flow Packets/s'].median()

print('Median of Flow Bytes/s: ', med_flow_bytes)
print('Median of Flow Packets/s: ', med_flow_packets)

Median of Flow Bytes/s:  4586.600756
Median of Flow Packets/s:  109.4759992228


In [36]:
# na를 중앙값으로 채우기
df['Flow Bytes/s'].fillna(med_flow_bytes, inplace = True)
df['Flow Packets/s'].fillna(med_flow_packets, inplace = True)

In [37]:
# 이름이 이상한 Label 바꾸기
attack_map = {
    'BENIGN': 'BENIGN',
    'DDoS': 'DDoS',
    'DoS Hulk': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'PortScan': 'Port Scan',
    'FTP-Patator': 'Brute Force',
    'SSH-Patator': 'Brute Force',
    'Bot': 'Bot',
    'Web Attack � Brute Force': 'Web Attack',
    'Web Attack � XSS': 'Web Attack',
    'Web Attack � Sql Injection': 'Web Attack',
    'Infiltration': 'Infiltration',
    'Heartbleed': 'Heartbleed'
}

In [38]:
# Label column 없애고 Attack Type로 변경
df['Attack Type'] = df['Label'].map(attack_map)
df.drop('Label', axis = 1, inplace = True)
df['Attack Type'].value_counts()

,count
Attack Type,
BENIGN,2273097
DoS,252661
Port Scan,158930
DDoS,128027
Brute Force,13835
Web Attack,2180
Bot,1966
Infiltration,36
Heartbleed,11


In [ ]:
df.to_csv('preprocessing_cicids2017.csv', index=False, encoding='utf-8')